### Training

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 \
python -m torch.distributed.launch --nproc_per_node=8 \
/root/ACCV2022/train.py \
--csv-dir autodl-tmp/ACCV_384_balance_fold.csv \
--config-name 'timm' \
--image-size 384 \
--batch-size 7 \
--num-workers 10 \
--init-lr 6e-5 \
--n-epochs 10 \
--cpkt_epoch 1 \
--n_batch_log 300 \
--warm_up_epochs 1 \
--fold 1

### Inference

In [ ]:
%cd ACCV2022

In [ ]:
from glob import glob
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import torch
import cv2
import albumentations as A
from build_model import XL_TIMM_Net, XL_DEIT_Net
from config_timm_deit_arc import get_config
config_deit_arc = get_config()
from config_timm_beit_384_sub import get_config
config_beit_sub = get_config()
from config_timm_beitv2_sub import get_config
config_beitv2_sub = get_config()
from config_timm_swin_sub import get_config
config_swin_sub = get_config()
from config_timm_beit_512_sub import get_config
config_beit512_sub = get_config()
from config_timm_swinv2_sub import get_config
config_swinv2_sub = get_config()
from config_timm_Deit_sub import get_config
config_Deit_sub = get_config()
from dataset import ACCV_test

model1 = XL_TIMM_Net(config_beit_sub, None)
state_dict = torch.load('../autodl-tmp/beit_new_wash_727_384.pth', map_location='cpu')['state_dict']
new_state_dict = {}
for k,v in state_dict.items():
    key = k.replace('module.','')
    new_state_dict[key]=v
model1.load_state_dict(new_state_dict, strict=True)
model1.cuda().eval()

model2 = XL_TIMM_Net(config_beit_sub, None)
state_dict = torch.load('../autodl-tmp/TIMM_best_731_beit_384.pth', map_location='cpu')['state_dict']
new_state_dict = {}
for k,v in state_dict.items():
    key = k.replace('module.','')
    new_state_dict[key]=v
model2.load_state_dict(new_state_dict, strict=True)
model2.cuda().eval()

model3 = XL_TIMM_Net(config_beit_sub, None)
state_dict = torch.load('../autodl-tmp/TIMM_best_730_beit_384.pth', map_location='cpu')['state_dict']
new_state_dict = {}
for k,v in state_dict.items():
    key = k.replace('module.','')
    new_state_dict[key]=v
model3.load_state_dict(new_state_dict, strict=True)
model3.cuda().eval()

model4 = XL_TIMM_Net(config_beit512_sub, None)
state_dict = torch.load('../autodl-tmp/beit_512_ep3.pth', map_location='cpu')['state_dict']
new_state_dict = {}
for k,v in state_dict.items():
    key = k.replace('module.','')
    new_state_dict[key]=v
model4.load_state_dict(new_state_dict, strict=True)
model4.cuda().eval()

model5 = XL_TIMM_Net(config_swinv2_sub, None)
state_dict = torch.load('../autodl-tmp/swinv2_384_epoch_2.pth', map_location='cpu')['state_dict']
new_state_dict = {}
for k,v in state_dict.items():
    key = k.replace('module.','')
    new_state_dict[key]=v
model5.load_state_dict(new_state_dict, strict=True)
model5.cuda().eval()

model6 = XL_TIMM_Net(config_deit_arc, None)
state_dict = torch.load('../autodl-tmp/deit_704_384_arc.pth', map_location='cpu')['state_dict']
new_state_dict = {}
for k,v in state_dict.items():
    key = k.replace('module.','')
    new_state_dict[key]=v
model6.load_state_dict(new_state_dict, strict=True)
model6.cuda().eval()

model7 = XL_TIMM_Net(config_beitv2_sub, None)
state_dict = torch.load('../autodl-tmp/TIMM_beitv2_224.pth', map_location='cpu')['state_dict']
new_state_dict = {}
for k,v in state_dict.items():
    key = k.replace('module.','')
    new_state_dict[key]=v
model7.load_state_dict(new_state_dict, strict=True)
model7.cuda().eval()

model8 = XL_TIMM_Net(config_swin_sub, None)
state_dict = torch.load('../autodl-tmp/swin_384_sub_epoch_3.pth', map_location='cpu')['state_dict']
new_state_dict = {}
for k,v in state_dict.items():
    key = k.replace('module.','')
    new_state_dict[key]=v
model8.load_state_dict(new_state_dict, strict=True)
model8.cuda().eval()

model9 = XL_DEIT_Net(config_Deit_sub, None)
state_dict = torch.load('../autodl-tmp/deit_384_cls_sub_epoch_3.pth', map_location='cpu')['state_dict']
new_state_dict = {}
for k,v in state_dict.items():
    key = k.replace('module.','')
    new_state_dict[key]=v
model9.load_state_dict(new_state_dict, strict=True)
model9.cuda().eval()
print('>>>>>>>>>> Load Successful! <<<<<<<<<<')

In [ ]:
import gc
from torchvision import transforms
test_files = glob('../autodl-tmp/testb/*')
df_dict = {'id': test_files}
df = pd.DataFrame(df_dict)

dataset_test = ACCV_test(df, 384)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=64, num_workers=16, shuffle=False, pin_memory=True)


pred_cls = []

test_files = glob('../autodl-tmp/testb/*')

for imgs in tqdm(dataloader_test):
    with torch.no_grad():
        imgs = imgs.cuda()
        imgs1 = transforms.functional.resize(imgs, size=[600, 600])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[384, 384])
        pred = 0
        pred += model1(imgs1).softmax(1)
        pred += model2(imgs1).softmax(1)
        pred += model3(imgs1).softmax(1)
        pred += 0.8*model9(imgs1).softmax(1)
        pred += 0.6*model5(imgs1).softmax(1)
        pred += 0.6*model6(imgs1).softmax(1)
        pred += 0.6*model8(imgs1).softmax(1)
    
        imgs1 = transforms.functional.resize(imgs, size=[480, 480])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[384, 384])
        pred += 0.3*model1(imgs1).softmax(1)
        pred += 0.3*model2(imgs1).softmax(1)
        pred += 0.3*model3(imgs1).softmax(1)
        pred += 0.24*model9(imgs1).softmax(1)
        pred += 0.18*model5(imgs1).softmax(1)
        pred += 0.18*model6(imgs1).softmax(1)
        pred += 0.18*model8(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[540, 540])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[384, 384])
        pred += 0.4*model1(imgs1).softmax(1)
        pred += 0.4*model2(imgs1).softmax(1)
        pred += 0.4*model3(imgs1).softmax(1)
        pred += 0.32*model9(imgs1).softmax(1)
        pred += 0.24*model5(imgs1).softmax(1)
        pred += 0.24*model6(imgs1).softmax(1)
        pred += 0.24*model8(imgs1).softmax(1)
    
        imgs1 = transforms.functional.resize(imgs, size=[660, 660])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[384, 384])
        pred += 0.4*model1(imgs1).softmax(1)
        pred += 0.4*model2(imgs1).softmax(1)
        pred += 0.4*model3(imgs1).softmax(1)
        pred += 0.32*model9(imgs1).softmax(1)
        pred += 0.24*model5(imgs1).softmax(1)
        pred += 0.24*model6(imgs1).softmax(1)
        pred += 0.24*model8(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[700, 700])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[384, 384])
        pred += 0.3*model1(imgs1).softmax(1)
        pred += 0.3*model2(imgs1).softmax(1)
        pred += 0.3*model3(imgs1).softmax(1)
        pred += 0.24*model9(imgs1).softmax(1)
        pred += 0.18*model5(imgs1).softmax(1)
        pred += 0.18*model6(imgs1).softmax(1)
        pred += 0.18*model8(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[400, 400])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[224, 224])
        pred += 0.3*model7(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[370, 370])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[224, 224])
        pred += 0.4*model7(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[350, 350])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[224, 224])
        pred += model7(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[330, 330])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[224, 224])
        pred += 0.4*model7(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[300, 300])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[224, 224])
        pred += 0.3*model7(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[700, 700])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[512, 512])
        pred += 0.5*model4(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[750, 750])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[512, 512])
        pred += 0.6*model4(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[800, 800])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[512, 512])
        pred += 1.4*model4(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[850, 850])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[512, 512])
        pred += 0.6*model4(imgs1).softmax(1)
        
        imgs1 = transforms.functional.resize(imgs, size=[900, 900])
        imgs1 = transforms.functional.center_crop(imgs1, output_size=[512, 512])
        pred += 0.5*model4(imgs1).softmax(1)
        
        pred_cls.extend(pred.argmax(1).cpu().detach().numpy())

for i in trange(len(test_files)):
    test_files[i] = test_files[i].split('/')[-1]
    pred_cls[i] = str(pred_cls[i]).zfill(4)

df_dict = {'id': test_files,
           'pred': pred_cls}

df = pd.DataFrame(df_dict)
df.to_csv('../autodl-tmp/pred_results.csv', index=False, header=False)
print(df.nunique())
df.head(5)